# XGBoost Model Development

**XGBoost** is an optimized distributed gradient boosting library designed to be highly efficient, flexible, and portable. It implements machine learning algorithms under the Gradient Boosting framework. XGBoost provides a parallel tree boosting (also known as GBDT, GBM) that solves many data science problems in a fast and accurate way.

### 1. Load the Datasets

In [1]:
import pandas as pd
from pathlib import Path

# Directory where the processed data is stored
data_path = Path("../processed_data")

# Load the training and validation datasets
X_train, X_val, y_train, y_val = (
    pd.read_csv(data_path / "X_train.csv"),
    pd.read_csv(data_path / "X_val.csv"),
    pd.read_csv(data_path / "y_train.csv"),
    pd.read_csv(data_path / "y_val.csv")
)

# Combine train and validation sets for robust K-Fold tuning
features = pd.concat([X_train, X_val], ignore_index=True)
targets = pd.concat([y_train, y_val], ignore_index=True)

# Display the shapes of the datasets
print(f"features shape: {features.shape}")
print(f"targets shape: {targets.shape}")

features shape: (2000, 157)
targets shape: (2000, 10)


### 2. Hyperparameter Tuning & Model Training

In [2]:
import optuna
import warnings
import numpy as np
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_percentage_error

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

# Define the objective function for Optuna
def objective(trial: optuna.Trial, X: pd.DataFrame, y: pd.Series):
    """
    Objective function for Optuna to minimize Mean Absolute Percentage Error (MAPE)
    using an XGBoost Regressor with K-Fold cross-validation.

    Parameters:
      trial (optuna.Trial): An Optuna trial object that suggests hyperparameters.
      X (pd.DataFrame): Feature matrix for training.
      y (pd.Series): Target variable for training.

    Returns:
      float: The mean absolute percentage error (MAPE) of the model on the validation set during cross-validation.
    """
    # Define the hyperparameter search space for XGBoost
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'mape',
        'early_stopping_round': 10,
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'random_state': 42,
        'n_jobs': -1
    }

    # Use K-Fold cross-validation to get a robust estimate of the model's performance
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    mape_scores = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        model = xgb.XGBRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
        preds = model.predict(X_val)
        mape_scores.append(mean_absolute_percentage_error(y_val, preds))

    return np.mean(mape_scores)

In [3]:
import joblib

# Define the directory for saving models and Optuna studies
model_dir = Path("../models/xgboost")
model_dir.mkdir(parents=True, exist_ok=True)

optuna_dir = Path("../optuna_db")
optuna_dir.mkdir(parents=True, exist_ok=True)
storage_name = f"sqlite:///{optuna_dir}/xgboost_studies.db"

# Dictionary to store the best models
best_models = {}

# Iterate over each target property to tune and train a model
for target in targets.columns:
    print(f"\n--- Tuning and Training for {target} ---\n")
    y = targets[target]

    # Create an Optuna study to find the best hyperparameters
    study = optuna.create_study(direction='minimize',
                                study_name='lightgbm-tuning-' + target,
                                storage=storage_name)
    study.optimize(lambda trial: objective(trial, features, y), n_trials=50)

    # Get the best hyperparameters
    best_params = study.best_params
    print(f"Best MAPE for {target}: {study.best_value}")
    print(f"Best hyperparameters for {target}: {best_params}")

    # Train the final model with the best hyperparameters on the entire training set
    final_model = xgb.XGBRegressor(**best_params, random_state=42, n_jobs=-1)
    final_model.fit(features, y)

    # Save the trained model to a file
    joblib.dump(final_model, f'{model_dir}/{target}_model.joblib')
    print(f"Saved best model for {target}")

    # Store the best model with its MAPE score in the dictionary
    best_models[target] = (final_model, study.best_value)


--- Tuning and Training for BlendProperty1 ---



[I 2025-07-18 18:02:10,912] A new study created in RDB with name: lightgbm-tuning-BlendProperty1
[I 2025-07-18 18:02:18,871] Trial 0 finished with value: 4.141636731702515 and parameters: {'n_estimators': 371, 'learning_rate': 0.12336954551514612, 'max_depth': 12, 'min_child_weight': 1, 'subsample': 0.6645881996196517, 'colsample_bytree': 0.9274202984276753, 'gamma': 0.9768356863092365}. Best is trial 0 with value: 4.141636731702515.
[I 2025-07-18 18:02:27,523] Trial 1 finished with value: 1.8327686129433811 and parameters: {'n_estimators': 599, 'learning_rate': 0.2710510703021149, 'max_depth': 4, 'min_child_weight': 4, 'subsample': 0.6027453482840737, 'colsample_bytree': 0.9304553850344833, 'gamma': 0.4868454381737741}. Best is trial 1 with value: 1.8327686129433811.
[I 2025-07-18 18:02:51,336] Trial 2 finished with value: 3.4797449080854412 and parameters: {'n_estimators': 995, 'learning_rate': 0.06564972684491026, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.853074398113560

Best MAPE for BlendProperty1: 1.1241262852486753
Best hyperparameters for BlendProperty1: {'n_estimators': 880, 'learning_rate': 0.22507387950939334, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.6978817111166078, 'colsample_bytree': 0.8463349997475814, 'gamma': 2.8690698326696795}


[I 2025-07-18 18:11:36,715] A new study created in RDB with name: lightgbm-tuning-BlendProperty2


Saved best model for BlendProperty1

--- Tuning and Training for BlendProperty2 ---



...

Best MAPE for BlendProperty9: 1.2869225268241347
Best hyperparameters for BlendProperty9: {'n_estimators': 774, 'learning_rate': 0.14749442928467016, 'max_depth': 8, 'min_child_weight': 7, 'subsample': 0.7304992131867065, 'colsample_bytree': 0.7151321380366973, 'gamma': 0.7974542821606588}


[I 2025-07-18 19:41:01,315] A new study created in RDB with name: lightgbm-tuning-BlendProperty10


Saved best model for BlendProperty9

--- Tuning and Training for BlendProperty10 ---



[I 2025-07-18 19:41:14,215] Trial 0 finished with value: 0.9983601362868069 and parameters: {'n_estimators': 910, 'learning_rate': 0.23283198153222062, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.6618314725628516, 'colsample_bytree': 0.9336520233255374, 'gamma': 1.7412321783481004}. Best is trial 0 with value: 0.9983601362868069.
[I 2025-07-18 19:41:25,109] Trial 1 finished with value: 1.3377526006046 and parameters: {'n_estimators': 773, 'learning_rate': 0.1975905326060107, 'max_depth': 8, 'min_child_weight': 4, 'subsample': 0.7333482346004172, 'colsample_bytree': 0.6427291964715055, 'gamma': 4.982229312136605}. Best is trial 0 with value: 0.9983601362868069.
[I 2025-07-18 19:41:40,271] Trial 2 finished with value: 0.8624816962546552 and parameters: {'n_estimators': 726, 'learning_rate': 0.09207896776720402, 'max_depth': 6, 'min_child_weight': 9, 'subsample': 0.7482231921331831, 'colsample_bytree': 0.662025698895869, 'gamma': 1.5594917071310883}. Best is trial 2 with value: 

Best MAPE for BlendProperty10: 0.597802886377014
Best hyperparameters for BlendProperty10: {'n_estimators': 140, 'learning_rate': 0.07885605063192688, 'max_depth': 3, 'min_child_weight': 5, 'subsample': 0.9812543671296039, 'colsample_bytree': 0.6765990082833855, 'gamma': 0.05835828780964476}
Saved best model for BlendProperty10


### 3. List the Best Models with MAPE Scores

In [4]:
print("--- BEST MODELS AND MAPE SCORES ---\n")
for target, (model, mape) in best_models.items():
    print(f"{target}: MAPE = {mape:.4f}, Model = {model}")
print("--- ALL MODELS SAVED TO DISK ---\n")
print(f"Models are saved in: {model_dir}")
print(f"Optuna studies are saved in: {optuna_dir}")

--- BEST MODELS AND MAPE SCORES ---

BlendProperty1: MAPE = 1.1241, Model = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8463349997475814, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, feature_weights=None,
             gamma=2.8690698326696795, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.22507387950939334,
             max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=880, n_jobs=-1,
             num_parallel_tree=None, ...)
BlendProperty2: MAPE = 0.7362, Model = XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylev